In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
copx =  yf.Ticker("COPX").history(period="max", interval="1d")# yf.Ticker("COPX").history(period="1y", interval="1h")
copx = copx.sort_values(by='Date')
copx['Daily Return'] = copx['Close'].pct_change()  
copx = copx.dropna()
copx

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Daily Return
0,2010-04-20 00:00:00-04:00,33.153598,33.153598,32.597329,32.664082,1933,0.0,0.0,0.0,NaN
1,2010-04-21 00:00:00-04:00,32.486067,32.708574,32.063303,32.152306,2900,0.0,0.0,0.0,-0.015668
2,2010-04-22 00:00:00-04:00,32.018798,32.018798,31.529282,31.974298,3600,0.0,0.0,0.0,-0.005536
3,2010-04-23 00:00:00-04:00,31.863057,32.085564,31.707302,32.085564,1100,0.0,0.0,0.0,0.003480
4,2010-04-26 00:00:00-04:00,33.287103,33.287103,32.041061,32.575077,3600,0.0,0.0,0.0,0.015256
...,...,...,...,...,...,...,...,...,...,...
3670,2024-11-15 00:00:00-05:00,41.590000,41.799999,41.049999,41.099998,1825300,0.0,0.0,0.0,-0.006767
3671,2024-11-18 00:00:00-05:00,41.200001,42.209999,41.169998,42.180000,1194200,0.0,0.0,0.0,0.026277
3672,2024-11-19 00:00:00-05:00,42.119999,42.959999,41.970001,42.900002,1055400,0.0,0.0,0.0,0.017070
3673,2024-11-20 00:00:00-05:00,42.529999,42.930000,42.459999,42.529999,1196600,0.0,0.0,0.0,-0.008625


First Quantum Minerals Ltd. (FM.TO)

Lundin Mining Corporation (LUN.TO)

Capstone Copper Corp. (CS.TO)

Boliden AB (BOL.ST)

Ivanhoe Mines Ltd. (IVN.TO)

Glencore plc (GLEN.L)

Southern Copper Corporation (SCCO)

Teck Resources Limited (TECK.B.TO)

Freeport-McMoRan Inc. (FCX)

BHP Group Limited (BHP.AX)

In [28]:
ticker = "FM.TO"
data =  yf.Ticker(ticker).history(period="max", interval="1d").reset_index()
data = data.sort_values(by='Date')
LAG_DAY = 1
data[f'{ticker} Daily Return [{LAG_DAY} day]'] = data['Close'].pct_change()  
data['Match to Date'] = data['Date'].shift(-LAG_DAY)
feature = data[['Match to Date', f'{ticker} Daily Return [{LAG_DAY} day]']]

In [29]:
df = pd.merge(copx, feature, left_on='Date', right_on='Match to Date', how='left')
df['High/Open(%)'] = df['High']/ df['Open']
df['Low/Open(%)'] = df['Low'] / df['Open']

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Daily Return,Match to Date,FM.TO Daily Return [1 day]
0,2010-04-20 00:00:00-04:00,33.153598,33.153598,32.597329,32.664082,1933,0.0,0.0,0.0,NaN,2010-04-20 00:00:00-04:00,0.015623
1,2010-04-21 00:00:00-04:00,32.486067,32.708574,32.063303,32.152306,2900,0.0,0.0,0.0,-0.015668,2010-04-21 00:00:00-04:00,0.016258
2,2010-04-22 00:00:00-04:00,32.018798,32.018798,31.529282,31.974298,3600,0.0,0.0,0.0,-0.005536,2010-04-22 00:00:00-04:00,-0.011445
3,2010-04-23 00:00:00-04:00,31.863057,32.085564,31.707302,32.085564,1100,0.0,0.0,0.0,0.003480,2010-04-23 00:00:00-04:00,-0.005478
4,2010-04-26 00:00:00-04:00,33.287103,33.287103,32.041061,32.575077,3600,0.0,0.0,0.0,0.015256,2010-04-26 00:00:00-04:00,0.013894
...,...,...,...,...,...,...,...,...,...,...,...,...
3670,2024-11-15 00:00:00-05:00,41.590000,41.799999,41.049999,41.099998,1825300,0.0,0.0,0.0,-0.006767,2024-11-15 00:00:00-05:00,0.002787
3671,2024-11-18 00:00:00-05:00,41.200001,42.209999,41.169998,42.180000,1194200,0.0,0.0,0.0,0.026277,2024-11-18 00:00:00-05:00,-0.021679
3672,2024-11-19 00:00:00-05:00,42.119999,42.959999,41.970001,42.900002,1055400,0.0,0.0,0.0,0.017070,2024-11-19 00:00:00-05:00,0.027273
3673,2024-11-20 00:00:00-05:00,42.529999,42.930000,42.459999,42.529999,1196600,0.0,0.0,0.0,-0.008625,2024-11-20 00:00:00-05:00,0.016040


In [ ]:
tickers = ['BOL.ST'] # 'FM.TO', 'LUN.TO', 'CS.TO', 
LAG_DAYS = [1, 2, 3, 4, 5]
for ticker in tickers:
    data =  yf.Ticker(ticker).history(period="max", interval="1d").reset_index()
    data['Date'] = data['Date'].dt.date
    data = data.sort_values(by='Date')
    for LAG_DAY in LAG_DAYS:
        feature1 = f'{ticker} Daily Return [{LAG_DAY} day]'
        feature2 = f'{ticker} High/Open(%) [{LAG_DAY} day]'
        data[feature1] = data['Close'].pct_change().shift(LAG_DAY)
        data[feature2] = (data['High'] / data['Open']).shift(LAG_DAY)
        
        features = data[['Date', feature1, feature2]]
        copx = pd.merge(copx, features, on='Date', how='left')
        
copx